# Gaussian process with Spike and Slab ARD prior - Stage 1 GP experiment
## Zero-temperature-MF VI algorithm (continuous spike) - non additive kernels

### (1) Importing and defining all required functions

In [1]:
"""
Importing in libraries for SGD-SS-GP
"""
# Requirements for algorithms
import numpy as np
import scipy.special as sp
from functools import partial
from scipy.spatial.distance import cdist

# Requirements for simulations/monitoring algorithms
import time
from IPython.display import clear_output
import inspect
from sklearn.metrics import roc_auc_score

# Requirements For plots and diagnostics
import matplotlib.pyplot as plt
import scipy.stats as sps
from mpl_toolkits import mplot3d

# Make plots inline
%matplotlib inline

"""
Importing algorithm functions
"""
import os
os.chdir('C:/Users/hughw/Documents/MSC project/GP algorithms/Master function files')
from GP_funcs_ZTMFSS import kernel_funcs
from GP_funcs_ZTMFSS import model_funcs
from GP_funcs_ZTMFSS import draw_GP
from GP_funcs_ZTMFSS import fit
from GP_funcs_ZTMFSS import diagnostics
from GP_funcs_ZTMFSS import simulations
from functools import partial
os.chdir('C:/Users/hughw/Documents/MSC project/Simulation results')

### (2) Setting simulation parameters and models

In [2]:
"""
Simulation controls
"""
# Simulation settings
n=300
ntest=100
p=100
q=5
corr=0.5 # used for GP draw
r2=2/3 # used for GP draw
lsmean=0.25
sigma2=1
ltrue=np.ones(p)*lsmean/q**0.5
strue=1

nruns = 100


# Model run settings
nmodels = 3
VS_threshs = [[0.1**2,0.1**1.5,0.1**1,0.1**0.5,0.1**0],
              [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99]]
iter_remove = [False,False,False,False,False,False,False]
sampling_strat = ["unif","unif", "unif", "unif", "unif", "unif", "unif"]
minibatch_size = [n,150,150,75,75,n,n]
GPtol=[1e-10, 1e-5,1e-5,1e-5,1e-5,1e-5,1e-5]
SS_GP=[False, True,True,True,True,True,True]
MC_pred = [False,False,False,False,False,False,False]
predict_selected = [False,False,False,False,False,False,False]
post_fit = [False,False,False,False,False,False,False]
train = [True,True,False,True, False, True,False] 
step = [0.01,0.025,0.025,0.025,0.025,0.0025,0.0025]
hyper_opt = [False,True,True,True,True,True,True]
model_select = [True,False,True,False,True,False,True]
post_var = [False,True,True,True,True,True,True]
model_weights = ["","elpd", "elpd","elpd", "elpd","elpd", "elpd"]
min_VBEM_iter = [1,5,5,5,5,5,5]
max_VBEM_iter = [1,10,10,10,10,10,10]
gp_iter = [500,100,100,100,100,100,100]
X_mult = [1,1,1,1,1,1,1]
opt = ["adam", "amsgrad", "amsgrad", "amsgrad", "amsgrad", "amsgrad", "amsgrad"]
l_init = 0.01
beta2=0.99
t = len(VS_threshs[0])
kern=kernel_funcs.gaussian
grad_kern=kernel_funcs.grad_gaussian
reg = 0.01
NN_pred = False
newsumgrads = False
VBtol=0.1/p
temp=1
v0=1e+4
v1=1e-4
postfit = "mixed"
# SEE BELOW FOR ARGVALS

# Storage objects
m=nmodels
Runtime=np.zeros((nruns, m))
Lambda = np.zeros((nruns, m, p))
L = np.zeros((nruns, m, p))
L1norm=np.zeros((nruns, m))
L2norm=np.zeros((nruns, m))
MSE_F=np.zeros((nruns, m))
MSE_Y=np.zeros((nruns,m))
Acc=np.zeros((nruns,m,t))
Weighted_Acc=np.zeros((nruns,m,t))
TPR=np.zeros((nruns,m,t))
TNR=np.zeros((nruns,m,t))
PPV=np.zeros((nruns,m,t))
NPV=np.zeros((nruns,m,t))
AUC=np.zeros((nruns,m))
MCC=np.zeros((nruns,m,t))

### (3) Drawing synthetic data, running algorithm, and displaying results

In [3]:
np.random.seed(8750)
runlist = np.random.choice(1000,nruns,False) # Choose 100 random trials
for run in range(len(runlist)):
    
    """
    Generating data and scaling data
    """
    lselect=[]
    np.random.seed(runlist[run]) # Fixing trial seed
    t=time.time()
    Y,F,X,e,lselect,strue,sigma,select=draw_GP.draw_GP_ARD_lm(n,ntest,p,q,sigma2,corr,strue,ltrue,plot_YX=True,kern=kern,cop=False,r2=r2)
    
    Y = Y.reshape(n+ntest,1)
    F = F.reshape(n+ntest,1)
    
    Y = (Y-Y[:n].mean())/Y[:n].var()**0.5
    X = (X-X[:n].mean(0))/X[:n].var(0)**0.5
    F = (F-F[:n].mean())/F[:n].var()**0.5

    # Getting training and test set
    ytest=Y[n:]
    Xtest=X[n:]
    ftest=F[n:]
    y=Y[:n]
    X=X[:n]
    f=F[:n]
    print("data generated")
    if lselect.any():
        print("Length-scales are: ",lselect[select])
    print("Noise variance is: ",sigma**2)
    print("Average data variance is: ", np.mean(np.var(X,0)))
    print("Time taken to draw data : ", time.time()-t)
    
    """
    Running algorithm
    """
    args = []
    arg_vals = []
    for i in range(nmodels):
        args.append(["seed","subsample", "sampling_strat", "min_VBEM_iter", "max_VBEM_iter", "GP_fit_tol", "VBEM_tol", "max_GP_fit_iter", "init_GP_iter", "iter_remove", "print_VBEM", "learn_rate", "optimisation", "X_mult", "final_prune"])
        arg_vals.append([0,minibatch_size[i], sampling_strat[i], min_VBEM_iter[i], max_VBEM_iter[i], GPtol[i], 0.1/p, gp_iter[i],gp_iter[i], iter_remove[i], False, step[i], opt[i], X_mult[i], True])

    test_algorithm = partial(diagnostics.get_pred_posterior_GP,reg = reg,kern = kernel_funcs.gaussian)

    Runtime[run], Lambda[run], L[run], L1norm[run], L2norm[run], MSE_F[run], MSE_Y[run], Acc[run], Weighted_Acc[run], TPR[run], TNR[run], PPV[run], NPV[run], AUC[run], MCC[run]= simulations.do_simulation_VBEMSSGP(
                               y, X, ftest, ytest, Xtest, q, algorithm_training = fit.VB_EM_GP_SS, algorithm_testing = test_algorithm, 
                               nmodels = m, args = args, arg_vals = arg_vals, post_fit = post_fit, SS_GP = SS_GP, post_var = post_var,
                               order_relevant_vars = False, order_irrelevant_vars = False, VS_threshs = VS_threshs, 
                               select = select, predict_selected = predict_selected, hyper_opt = hyper_opt, hyper_arg = ["v0","v1"],
                                hyper_vals = [1e+4*2**np.linspace(np.log2(100),-np.log2(100),11),1e-4*2**np.linspace(np.log2(100),-np.log2(100),11)], ltrue=lselect, MC_pred = MC_pred, model_select = model_select, post_fit_subsample=n, train = train,
                                model_weighting = model_weights)
    
    print("RUN {0}".format(run))
    print("Runtime mean is:", Runtime[:run+1].mean(0))
    print("Weighted accuracy mean is:", Weighted_Acc[:run].mean(0))
    print("TPR mean is:", TPR[:run+1].mean(0))
    print("PPV mean is:", PPV[:run+1].mean(0))
    print("MCC mean is:", MCC[:run+1].mean(0))
    print("L1norm mean is:", L1norm[:run+1].mean(0))
    print("L2norm mean is:", L2norm[:run+1].mean(0))
    print("MSE_F mean is:", MSE_F[:run+1].mean(0))
    print("MSE_Y mean is:", MSE_Y[:run+1].mean(0), "\n")
    

R2= 0.6650981093248958
data generated
Length-scales are:  [0.1118034 0.1118034 0.1118034 0.1118034 0.1118034]
Noise variance is:  0.027304068902007185
Average data variance is:  1.0
Time taken to draw data :  0.0727999210357666
run time is : 36.41157627105713


C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:2035: RuntimeWarning: invalid value encountered in true_divide
  MCC[j,i]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))


run time is : 19.008038997650146
run time is : 19.279218196868896
run time is : 19.154767751693726
run time is : 17.16808247566223
run time is : 15.293095588684082
run time is : 11.521149635314941
run time is : 9.606305122375488
run time is : 20.912068843841553
run time is : 19.929807662963867


C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T
<ipython-input-3-4ce5220f69db>:55: RuntimeWarning: Mean of empty slice.
  print("Weighted accuracy mean is:", Weighted_Acc[:run].mean(0))
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


RUN 0
Runtime mean is: [ 36.43351769 164.57260728 164.57620716]
Weighted accuracy mean is: [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
TPR mean is: [[1.  1.  0.8 0.  0. ]
 [1.  1.  1.  1.  0. ]
 [1.  1.  1.  1.  0. ]]
PPV mean is: [[0.29411765 0.29411765 0.57142857 0.                nan]
 [0.45454545 0.71428571 1.         1.         0.        ]
 [0.38461538 0.38461538 0.38461538 0.38461538 0.        ]]
MCC mean is: [[ 0.50691808  0.50691808  0.65638034 -0.02305715  0.        ]
 [ 0.65256212  0.83621057  1.          1.          0.        ]
 [ 0.59348692  0.59348692  0.59348692  0.59348692  0.        ]]
L1norm mean is: [1.24256787 0.66025893 0.88119474]
L2norm mean is: [0.43193128 0.21202597 0.27527634]
MSE_F mean is: [0.30318766 0.12911659 0.1893365 ]
MSE_Y mean is: [0.49840901 0.45543769 0.47436435] 

R2= 0.6721101071837331
data generated
Length-scales are:  [0.1118034 0.1118034 0.1118034 0.1118034 0.1118034]
Noise variance is:  0.025248529499040346
Average da

...

C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T


RUN 98
Runtime mean is: [ 39.47615108 142.9345889  142.91468526]
Weighted accuracy mean is: [[0.84801289 0.84973147 0.85843179 0.54844253 0.5       ]
 [0.88839957 0.90494092 0.90580021 0.90456498 0.        ]
 [0.88485499 0.8849087  0.88496241 0.88501611 0.        ]]
TPR mean is: [[0.90909091 0.90505051 0.77575758 0.1010101  0.        ]
 [0.86464646 0.85050505 0.84242424 0.82020202 0.        ]
 [0.87676768 0.87676768 0.87676768 0.87676768 0.        ]]
PPV mean is: [[0.20144353 0.20618899 0.48917748        nan        nan]
 [0.37289914 0.58932252 0.66776333 0.83610309 0.        ]
 [0.32068906 0.3208376  0.3210041  0.32113701 0.        ]]
MCC mean is: [[0.36716893 0.37160787 0.57052071 0.15379926 0.        ]
 [0.52661849 0.67641906 0.7197077  0.81009194 0.        ]
 [0.48708125 0.48724015 0.48740665 0.48755668 0.        ]]
L1norm mean is: [2.33293063 1.04544944 1.14052966]
L2norm mean is: [0.56831927 0.32449732 0.345282  ]
MSE_F mean is: [0.33176723 0.15920132 0.19105828]
MSE_Y mean is: [0

C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:2035: RuntimeWarning: invalid value encountered in true_divide
  MCC[j,i]=(TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))


run time is : 14.357285022735596
run time is : 14.362865686416626
run time is : 14.365272045135498
run time is : 14.350525856018066
run time is : 10.06474757194519
run time is : 10.041321277618408
run time is : 8.613930940628052
run time is : 7.2021660804748535
run time is : 14.374002456665039
run time is : 14.373491764068604
RUN 99
Runtime mean is: [ 39.49132134 142.80600554 142.78588358]
Weighted accuracy mean is: [[0.84885699 0.84960128 0.85762892 0.54795322 0.5       ]
 [0.88798511 0.90473153 0.90558214 0.90451887 0.        ]
 [0.88431685 0.88437002 0.88442318 0.88447634 0.        ]]
TPR mean is: [[0.908 0.904 0.776 0.1   0.   ]
 [0.864 0.85  0.842 0.82  0.   ]
 [0.876 0.876 0.876 0.876 0.   ]]
PPV mean is: [[0.20109576 0.20579376 0.49228571        nan        nan]
 [0.37167015 0.58650622 0.66416262 0.83774206 0.        ]
 [0.31948217 0.31962923 0.31979406 0.31992564 0.        ]]
MCC mean is: [[0.36650538 0.37089994 0.57271024 0.15226127 0.        ]
 [0.52535731 0.6742254  0.7170811

C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T


In [6]:
namelist = ["Runtime", "MSE_F", "MSE_Y", "Acc", "Weighted_Acc", "TPR", "TNR", "PPV", "NPV", "AUC", "MCC"]
objlist = [Runtime, MSE_F, MSE_Y, Acc, Weighted_Acc, TPR, TNR, PPV, NPV, AUC, MCC]
#iters = np.random.choice(1000,100,False)
iters = np.linspace(0,99,100).astype(int)

for i in range(len(objlist)):
    print("Mean {0} is:".format(namelist[i]), np.mean(objlist[i][iters],0))

print("\n")
for i in range(len(objlist)):
    print("Median {0} is:".format(namelist[i]), np.median(objlist[i][iters],0))

print("\n")
quant = 0.25
for i in range(len(objlist)):
    if namelist[i] in ["Runtime", "MSE_F", "MSE_Y"]:
        print("{1} quantile {0} is:".format(namelist[i], quant), np.quantile(objlist[i][iters],1-quant,0))
    else:
        print("{1} quantile {0} is:".format(namelist[i], quant), np.quantile(objlist[i][iters],quant,0))

Mean Runtime is: [ 39.49132134 142.80600554 142.78588358]
Mean MSE_F is: [0.33092817 0.15938383 0.19147841]
Mean MSE_Y is: [0.49066672 0.4014303  0.41099478]
Mean Acc is: [[0.7946 0.7996 0.9318 0.9502 0.95  ]
 [0.9086 0.953  0.9618 0.9805 0.    ]
 [0.8905 0.8906 0.8907 0.8908 0.    ]]
Mean Weighted_Acc is: [[0.84831579 0.84905263 0.858      0.54747368 0.5       ]
 [0.88747368 0.90421053 0.90505263 0.90447368 0.        ]
 [0.88363158 0.88368421 0.88373684 0.88378947 0.        ]]
Mean TPR is: [[0.908 0.904 0.776 0.1   0.   ]
 [0.864 0.85  0.842 0.82  0.   ]
 [0.876 0.876 0.876 0.876 0.   ]]
Mean TNR is: [[0.78863158 0.79410526 0.94       0.99494737 1.        ]
 [0.91094737 0.95842105 0.96810526 0.98894737 0.        ]
 [0.89126316 0.89136842 0.89147368 0.89157895 0.        ]]
Mean PPV is: [[0.20109576 0.20579376 0.49228571        nan        nan]
 [0.37167015 0.58650622 0.66416262 0.83774206 0.        ]
 [0.31948217 0.31962923 0.31979406 0.31992564 0.        ]]
Mean NPV is: [[0.99389811 0.

In [7]:
from datetime import date
Output = {"Runtime" : Runtime, "Lambda" : Lambda, "L" : L, "L1norm" : L1norm, "L2norm" : L2norm, "MSE_F" : MSE_F
        , "MSE_Y" : MSE_Y, "Acc" : Acc, "Weighted_Acc" : Weighted_Acc, "TPR" :TPR, "TNR" : TNR, "PPV" : PPV, "NPV" : NPV, "AUC" : AUC, "MCC" : MCC}
String = "Stage1_ZTnoremove{13}_{0}_{1}_lsmean={12}_r2={14}_l0={2}_b2={3}_step={4}_MCpred={5}_n={6}_p={7}_q={8}_kern={9}_runs={10}_start={11}".format(
    "GP", date.today(), l_init, beta2, step[0], MC_pred[0],n,p,q,str(kern)[23:28], nruns, "random", lsmean, minibatch_size, r2)
np.save(String, Output) # saving